In [2]:
import torch
import torch.nn as nn
import pandas as pd
import os
import cv2
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [3]:
class RBM(nn.Module):
    def __init__(self, n_vis, n_hid):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(n_hid, n_vis))
        self.v_bias = nn.Parameter(torch.zeros(n_vis))
        self.h_bias = nn.Parameter(torch.zeros(n_hid))

    def sample_h(self, v):
        p_h_given_v = torch.sigmoid(0.01 * torch.matmul(v, self.W.t()) + self.h_bias)
        return torch.bernoulli(p_h_given_v)

    def sample_v(self, h):
        p_h_given_h = torch.sigmoid(0.01 * torch.matmul(h, self.W.t()) + self.v_bias)
        # p_v_given_h = torch.sigmoid(torch.matmul(h, self.W) + self.v_bias)
        return torch.bernoulli(p_v_given_h)

    def forward(self, v):
        h = self.sample_h(v)
        v_recon = self.sample_v(h)
        return h, v_recon

    def free_energy(self, v):
        vbias_term = torch.matmul(v, self.v_bias)
        hidden_term = torch.sum(torch.log(1 + torch.exp(torch.matmul(v, self.W.t()) + self.h_bias)), dim=1)
        return -hidden_term - vbias_term


In [63]:
# Read training data
train_labels = pd.read_excel('../mini_dataset/Training_Set/train_labels.xlsx')
# Read test data
test_labels = pd.read_csv('../mini_dataset/Test_Set/test_labels.csv')
# Read validation data
validation_labels = pd.read_csv('../mini_dataset/Validation_Set/validation_labels.csv')


train_labels = train_labels.drop('ID', axis=1)
train_labels = train_labels.drop('Disease_Risk', axis=1)

Y_train = torch.tensor(train_labels.values)


path = '../mini_dataset/Training_Set/train_images/'
images = []

for file in os.listdir(path):
    if file.endswith('.png'):
        image = cv2.imread(os.path.join(path, file))
        image = cv2.resize(image, (28, 28))
        image = (image - np.mean(image)) / np.std(image)
        image = torch.from_numpy(image)
        images.append(image)

X_train = torch.stack(images)


In [64]:
# Define hyperparameters
batch_size = 32
learning_rate = 0.001# source: https://github.com/AmanPriyanshu/Deep-Belief-Networks-in-PyTorch/blob/main/RBM.py
num_epochs = 10

# Create data loader
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32))
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create RBM instance and optimizer
rbm = RBM(n_vis=28*28*3, n_hid=128)
optimizer = torch.optim.Adam(rbm.parameters(), lr=learning_rate)

# Train RBM
for epoch in range(num_epochs):
    total_loss = 0
    for batch in data_loader:
        v = batch[0].view(batch[0].size(0), -1)
        h, v_recon = rbm(v)
        loss = torch.mean(rbm.free_energy(v) - rbm.free_energy(v_recon))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * v.size(0)
    print("Epoch {} loss: {:.4f}".format(epoch+1, total_loss/len(data_loader.dataset)))

/tmp/ipykernel_9227/1222204941.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(Y_train, dtype=torch.float32))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x128 and 2352x128)